# Monarch Watch Data all taggings GBIF

In [98]:
#  Import and read
import pandas as pd 
import numpy as np
import json
# Sue Mottet 2-11-2022

In [99]:
MWatch_Tagged_df = pd.read_csv("MWatch_occurrences_1996_2000.csv")
MWatch_Tagged_df.head()

,id,modified,language,license,institutionID,collectionID,institutionCode,collectionCode,datasetName,basisOfRecord,...,decimalLatitude,decimalLongitude,geodeticDatum,georeferenceRemarks,taxonID,nameAccordingToID,scientificName,kingdom,vernacularName,taxonRemarks
0,MW0000001,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus
1,MW0000002,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus
2,MW0000003,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus
3,MW0000004,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus
4,MW0000005,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,38.61429,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus


In [100]:
MWatch_Tagged_df.columns.tolist()

['id',
 'modified',
 'language',
 'license',
 'institutionID',
 'collectionID',
 'institutionCode',
 'collectionCode',
 'datasetName',
 'basisOfRecord',
 'occurrenceID',
 'catalogNumber',
 'recordedBy',
 'occurrenceStatus',
 'associatedReferences',
 'eventDate',
 'year',
 'country',
 'stateProvince',
 'verbatimLocality',
 'decimalLatitude',
 'decimalLongitude',
 'geodeticDatum',
 'georeferenceRemarks',
 'taxonID',
 'nameAccordingToID',
 'scientificName',
 'kingdom',
 'vernacularName',
 'taxonRemarks']

In [101]:
def df_to_geojson(df, properties, lat='decimalLatitude', lon='decimalLongitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [102]:
MWatch_Tagged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363582 entries, 0 to 363581
Data columns (total 30 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    363582 non-null  object 
 1   modified              363582 non-null  object 
 2   language              363582 non-null  object 
 3   license               363582 non-null  object 
 4   institutionID         363582 non-null  object 
 5   collectionID          363582 non-null  object 
 6   institutionCode       363582 non-null  object 
 7   collectionCode        363582 non-null  object 
 8   datasetName           363582 non-null  object 
 9   basisOfRecord         363582 non-null  object 
 10  occurrenceID          363582 non-null  object 
 11  catalogNumber         363582 non-null  object 
 12  recordedBy            363582 non-null  object 
 13  occurrenceStatus      363582 non-null  object 
 14  associatedReferences  363582 non-null  object 
 15  

In [103]:
MWatch_Tagged_df.describe()

,year,decimalLatitude,decimalLongitude,taxonID
count,363582.000000,363582.000000,363582.000000,363582.0
mean,1997.858071,39.694501,-90.112869,117273.0
std,1.763730,3.769477,8.696296,0.0
min,1992.000000,25.721490,-118.997683,117273.0
25%,1997.000000,38.203070,-96.731103,117273.0
50%,1998.000000,39.216660,-93.664964,117273.0
75%,1999.000000,42.213930,-82.491268,117273.0
max,2000.000000,49.895140,-67.958850,117273.0


In [104]:
print(MWatch_Tagged_df.shape)

(363582, 30)


In [105]:
# Determine the number of unique values in each column.
for col in MWatch_Tagged_df:
  print(MWatch_Tagged_df[col].unique())

['MW0000001' 'MW0000002' 'MW0000003' ... 'MW0363580' 'MW0363581'
 'MW0363582']
['7/30/2019']
['English']
['https://creativecommons.org/publicdomain/zero/1.0/legalcode']
['https://bison.usgs.gov/']
['https://bison.usgs.gov/ipt/resource?r=monarchwatch']
['BISON']
['Monarch Watch']
['Monarch Watch']
['humanobservation']
['MW0000001' 'MW0000002' 'MW0000003' ... 'MW0363580' 'MW0363581'
 'MW0363582']
['MW0000001' 'MW0000002' 'MW0000003' ... 'MW0363580' 'MW0363581'
 'MW0363582']
['Monarch Watch']
['present']
['Monarch Watch Tagging Data, https://monarchwatch.org/tagdata']
['8/9/1992' '8/10/1992' '9/1/1992' ... '12/29/2000' '12/30/2000'
 '12/31/2000']
[1992 1993 1994 1995 1996 1997 1998 1999 2000]
['US' 'CA']
['MD' 'MO' 'KS' 'MI' 'KY' 'CT' 'NJ' 'CO' 'TX' 'OH' 'NY' 'MA' 'GA' 'IN'
 'PA' 'NC' 'MN' 'ON' 'NE' 'OK' 'FL' 'NH' 'VT' 'IA' 'IL' 'QC' 'WI' 'AZ'
 'VA' 'ME' 'WV' 'TN' 'SD' 'LA' 'RI' 'DE' 'AL' 'NM' 'AR' 'SC' 'ND' 'DC'
 'MS' 'MB' 'WY' 'BC' 'UT' 'ID']
['City:PLUM POINT' 'City:INDEPENDENCE' 'City

In [106]:
# Determine the number of unique values in each column.
for col in MWatch_Tagged_df:
  print(MWatch_Tagged_df[col].value_counts())

MW0000001    1
MW0242418    1
MW0242394    1
MW0242393    1
MW0242392    1
            ..
MW0121193    1
MW0121192    1
MW0121191    1
MW0121190    1
MW0363582    1
Name: id, Length: 363582, dtype: int64
7/30/2019    363582
Name: modified, dtype: int64
English    363582
Name: language, dtype: int64
https://creativecommons.org/publicdomain/zero/1.0/legalcode    363582
Name: license, dtype: int64
https://bison.usgs.gov/    363582
Name: institutionID, dtype: int64
https://bison.usgs.gov/ipt/resource?r=monarchwatch    363582
Name: collectionID, dtype: int64
BISON    363582
Name: institutionCode, dtype: int64
Monarch Watch    363582
Name: collectionCode, dtype: int64
Monarch Watch    363582
Name: datasetName, dtype: int64
humanobservation    363582
Name: basisOfRecord, dtype: int64
MW0000001    1
MW0242418    1
MW0242394    1
MW0242393    1
MW0242392    1
            ..
MW0121193    1
MW0121192    1
MW0121191    1
MW0121190    1
MW0363582    1
Name: occurrenceID, Length: 363582, dtype: int6

In [107]:
# Find null values
for column in MWatch_Tagged_df.columns:
   print(f"Column {column} has {MWatch_Tagged_df[column].isnull().sum()} null values")

Column id has 0 null values
Column modified has 0 null values
Column language has 0 null values
Column license has 0 null values
Column institutionID has 0 null values
Column collectionID has 0 null values
Column institutionCode has 0 null values
Column collectionCode has 0 null values
Column datasetName has 0 null values
Column basisOfRecord has 0 null values
Column occurrenceID has 0 null values
Column catalogNumber has 0 null values
Column recordedBy has 0 null values
Column occurrenceStatus has 0 null values
Column associatedReferences has 0 null values
Column eventDate has 0 null values
Column year has 0 null values
Column country has 0 null values
Column stateProvince has 0 null values
Column verbatimLocality has 0 null values
Column decimalLatitude has 0 null values
Column decimalLongitude has 0 null values
Column geodeticDatum has 0 null values
Column georeferenceRemarks has 0 null values
Column taxonID has 0 null values
Column nameAccordingToID has 0 null values
Column scienti

In [108]:
# Find duplicate entries
print(f"Duplicate entries: {MWatch_Tagged_df.duplicated().sum()}")

Duplicate entries: 0


# Format file for geoJSON

In [109]:
MWatch_Tagged_df['species'] = 'Monarch Tagged'

In [110]:
MWatch_Tagged_df.head()

,id,modified,language,license,institutionID,collectionID,institutionCode,collectionCode,datasetName,basisOfRecord,...,decimalLongitude,geodeticDatum,georeferenceRemarks,taxonID,nameAccordingToID,scientificName,kingdom,vernacularName,taxonRemarks,species
0,MW0000001,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged
1,MW0000002,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged
2,MW0000003,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged
3,MW0000004,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged
4,MW0000005,7/30/2019,English,https://creativecommons.org/publicdomain/zero/...,https://bison.usgs.gov/,https://bison.usgs.gov/ipt/resource?r=monarchw...,BISON,Monarch Watch,Monarch Watch,humanobservation,...,-76.512176,WGS84,centroid (type and size unkown),117273,Integrated Taxonomic Information System (ITIS)...,Danaus plexippus,Animalia,Monarch butterfly,Danaus plexippus,Monarch Tagged


In [111]:
MWatch_Tagged_df.drop(['modified', 'language', 'license', 'institutionID', 'collectionID', 'institutionCode', 'collectionCode', 'datasetName', 'basisOfRecord', 'occurrenceID', 'catalogNumber', 'recordedBy', 'occurrenceStatus', 'associatedReferences', 'verbatimLocality', 'geodeticDatum', 'georeferenceRemarks', 'taxonID', 'nameAccordingToID','scientificName','kingdom', 'vernacularName','taxonRemarks'], axis=1, inplace=True)
MWatch_Tagged_df.head()

,id,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species
0,MW0000001,8/9/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
1,MW0000002,8/9/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
2,MW0000003,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
3,MW0000004,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
4,MW0000005,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged


In [112]:
cols = ['eventDate', 'species','country', 'stateProvince', 'decimalLatitude', 'decimalLongitude', 'number']

In [113]:
MWatch_Tagged_df.shape

(363582, 8)

In [114]:
# MWatch_Tagged_df.replace(np.nan,'',regex=True)

In [115]:
MWatch_Tagged_df.shape

(363582, 8)

In [116]:
MWatch_Tagged_df

,id,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species
0,MW0000001,8/9/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
1,MW0000002,8/9/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
2,MW0000003,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
3,MW0000004,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
4,MW0000005,8/10/1992,1992,US,MD,38.61429,-76.512176,Monarch Tagged
...,...,...,...,...,...,...,...,...
363577,MW0363578,12/30/2000,2000,US,TX,29.50745,-95.094930,Monarch Tagged
363578,MW0363579,12/31/2000,2000,US,TX,29.50745,-95.094930,Monarch Tagged
363579,MW0363580,12/31/2000,2000,US,TX,29.50745,-95.094930,Monarch Tagged
363580,MW0363581,12/31/2000,2000,US,TX,29.50745,-95.094930,Monarch Tagged


In [117]:
# MWatch_Tagged_df['column name'] = df['column name'].replace(['old value'],'new value')

In [118]:
MWatch_Tagged_df =MWatch_Tagged_df.groupby(['eventDate', 'year', 'country', 'stateProvince', 'decimalLatitude', 'decimalLongitude', 'species']).size().reset_index()
MWatch_Tagged_df.head()

,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species,0
0,1/1/2000,2000,US,FL,28.01863,-82.112864,Monarch Tagged,1
1,1/1/2000,2000,US,FL,28.53834,-81.379237,Monarch Tagged,1
2,1/10/1998,1998,US,KS,37.58613,-99.475674,Monarch Tagged,9
3,1/10/1998,1998,US,TX,25.90175,-97.497484,Monarch Tagged,1
4,1/10/1998,1998,US,TX,29.76043,-95.369803,Monarch Tagged,7


In [119]:
MWatch_Tagged_df.rename( columns={0:'number'} ,inplace=True)
MWatch_Tagged_df.head()

,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species,number
0,1/1/2000,2000,US,FL,28.01863,-82.112864,Monarch Tagged,1
1,1/1/2000,2000,US,FL,28.53834,-81.379237,Monarch Tagged,1
2,1/10/1998,1998,US,KS,37.58613,-99.475674,Monarch Tagged,9
3,1/10/1998,1998,US,TX,25.90175,-97.497484,Monarch Tagged,1
4,1/10/1998,1998,US,TX,29.76043,-95.369803,Monarch Tagged,7


In [120]:
MWatch_Tagged_year_df = MWatch_Tagged_df
# MWatch_Tagged_year_df['years'] = pd.DatetimeIndex(MWatch_Tagged_year_df['eventDate']).year

# Filter Tagging Data By Year - 1998

In [121]:
# filtering by year
MWatch_Tagged_1998_df = MWatch_Tagged_df[MWatch_Tagged_df['year']==1998]
MWatch_Tagged_1998_df.head()

,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species,number
2,1/10/1998,1998,US,KS,37.58613,-99.475674,Monarch Tagged,9
3,1/10/1998,1998,US,TX,25.90175,-97.497484,Monarch Tagged,1
4,1/10/1998,1998,US,TX,29.76043,-95.369803,Monarch Tagged,7
7,1/11/1998,1998,US,TX,29.76043,-95.369803,Monarch Tagged,6
11,1/12/1998,1998,US,TX,29.76043,-95.369803,Monarch Tagged,4


In [122]:
MWatch_Tagged_1998_df['eventDate']

2        1/10/1998
3        1/10/1998
4        1/10/1998
7        1/11/1998
11       1/12/1998
           ...    
39697     9/9/1998
39698     9/9/1998
39699     9/9/1998
39700     9/9/1998
39701     9/9/1998
Name: eventDate, Length: 6533, dtype: object

In [123]:
MWatch_Tagged_1998_df.shape

(6533, 8)

In [124]:
MWatch_Tagged_1998_df = pd.DataFrame(MWatch_Tagged_1998_df)
MWatch_Tagged_1998_df['month'] = pd.DatetimeIndex(MWatch_Tagged_1998_df['eventDate']).month

In [125]:
MWatch_Tagged_1998_df.head()

,eventDate,year,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
2,1/10/1998,1998,US,KS,37.58613,-99.475674,Monarch Tagged,9,1
3,1/10/1998,1998,US,TX,25.90175,-97.497484,Monarch Tagged,1,1
4,1/10/1998,1998,US,TX,29.76043,-95.369803,Monarch Tagged,7,1
7,1/11/1998,1998,US,TX,29.76043,-95.369803,Monarch Tagged,6,1
11,1/12/1998,1998,US,TX,29.76043,-95.369803,Monarch Tagged,4,1


In [126]:
MWatch_Tagged_1998_df.drop(['year'], axis=1, inplace=True)

In [127]:
MWatch_Tagged_1998_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
2,1/10/1998,US,KS,37.58613,-99.475674,Monarch Tagged,9,1
3,1/10/1998,US,TX,25.90175,-97.497484,Monarch Tagged,1,1
4,1/10/1998,US,TX,29.76043,-95.369803,Monarch Tagged,7,1
7,1/11/1998,US,TX,29.76043,-95.369803,Monarch Tagged,6,1
11,1/12/1998,US,TX,29.76043,-95.369803,Monarch Tagged,4,1


## January

In [128]:
MWatch_Tagged_1998_month_df =pd.DataFrame(MWatch_Tagged_1998_df)
MWatch_Tagged_1998_1_df = MWatch_Tagged_1998_month_df[MWatch_Tagged_1998_month_df['month']==1]
MWatch_Tagged_1998_1_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
2,1/10/1998,US,KS,37.58613,-99.475674,Monarch Tagged,9,1
3,1/10/1998,US,TX,25.90175,-97.497484,Monarch Tagged,1,1
4,1/10/1998,US,TX,29.76043,-95.369803,Monarch Tagged,7,1
7,1/11/1998,US,TX,29.76043,-95.369803,Monarch Tagged,6,1
11,1/12/1998,US,TX,29.76043,-95.369803,Monarch Tagged,4,1


In [129]:
MWatch_Tagged_1998_1_df.shape

(37, 8)

In [130]:
MWatch_Tagged_1998_1_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
2,1/10/1998,US,KS,37.58613,-99.475674,Monarch Tagged,9,1
3,1/10/1998,US,TX,25.90175,-97.497484,Monarch Tagged,1,1
4,1/10/1998,US,TX,29.76043,-95.369803,Monarch Tagged,7,1
7,1/11/1998,US,TX,29.76043,-95.369803,Monarch Tagged,6,1
11,1/12/1998,US,TX,29.76043,-95.369803,Monarch Tagged,4,1


In [131]:
MWatch_Tagged_1998_1_df = pd.DataFrame(MWatch_Tagged_1998_1_df)
MWatch_Tagged_1998_1_df.drop(['month'], axis=1, inplace=True)

In [132]:
MWatch_Tagged_1998_1_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number
2,1/10/1998,US,KS,37.58613,-99.475674,Monarch Tagged,9
3,1/10/1998,US,TX,25.90175,-97.497484,Monarch Tagged,1
4,1/10/1998,US,TX,29.76043,-95.369803,Monarch Tagged,7
7,1/11/1998,US,TX,29.76043,-95.369803,Monarch Tagged,6
11,1/12/1998,US,TX,29.76043,-95.369803,Monarch Tagged,4


In [133]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1998_1_df['decimalLatitude'] = MWatch_Tagged_1998_1_df['decimalLatitude'].astype(float)
MWatch_Tagged_1998_1_df['decimalLongitude'] = MWatch_Tagged_1998_1_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [134]:
geojson = df_to_geojson(MWatch_Tagged_1998_1_df, cols)

In [135]:
file = open("MWatch_Tagged_1998_1.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## February

In [136]:
MWatch_Tagged_1998_month_df =pd.DataFrame(MWatch_Tagged_1998_df)
MWatch_Tagged_1998_2_df = MWatch_Tagged_1998_month_df[MWatch_Tagged_1998_month_df['month']==2]
MWatch_Tagged_1998_2_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11535,2/12/1998,US,TX,28.80527,-97.003598,Monarch Tagged,1,2
11539,2/16/1998,US,TX,28.80527,-97.003598,Monarch Tagged,2,2
11541,2/18/1998,US,FL,28.53834,-81.379237,Monarch Tagged,1,2
11556,2/3/1998,US,TX,28.80527,-97.003598,Monarch Tagged,2,2
11566,2/8/1998,US,TX,28.80527,-97.003598,Monarch Tagged,2,2


In [137]:
MWatch_Tagged_1998_2_df.shape

(6, 8)

In [138]:
MWatch_Tagged_1998_2_df = pd.DataFrame(MWatch_Tagged_1998_2_df)
MWatch_Tagged_1998_2_df.drop(['month'], axis=1, inplace=True)

In [139]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1998_2_df['decimalLatitude'] = MWatch_Tagged_1998_2_df['decimalLatitude'].astype(float)
MWatch_Tagged_1998_2_df['decimalLongitude'] = MWatch_Tagged_1998_2_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [140]:
geojson = df_to_geojson(MWatch_Tagged_1998_2_df, cols)

In [141]:
file = open("MWatch_Tagged_1998_2.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## March

In [142]:
MWatch_Tagged_1998_month_df =pd.DataFrame(MWatch_Tagged_1998_df)
MWatch_Tagged_1998_3_df = MWatch_Tagged_1998_month_df[MWatch_Tagged_1998_month_df['month']==3]
MWatch_Tagged_1998_3_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11604,3/27/1998,US,NC,35.43619,-83.359952,Monarch Tagged,6,3
11614,3/30/1998,US,TX,32.50070,-94.740489,Monarch Tagged,1,3
11618,3/4/1998,US,TX,27.80058,-97.396381,Monarch Tagged,1,3
11631,3/7/1998,US,TX,27.80058,-97.396381,Monarch Tagged,1,3
11639,3/9/1998,US,TX,27.80058,-97.396381,Monarch Tagged,1,3


In [143]:
MWatch_Tagged_1998_3_df.shape

(7, 8)

In [144]:
MWatch_Tagged_1998_3_df = pd.DataFrame(MWatch_Tagged_1998_3_df)
MWatch_Tagged_1998_3_df.drop(['month'], axis=1, inplace=True)

In [145]:
MWatch_Tagged_1998_3_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number
11593,3/21/1998,US,TX,28.80527,-97.003598,Monarch Tagged,1
11601,3/26/1998,US,TX,27.80058,-97.396381,Monarch Tagged,1
11604,3/27/1998,US,NC,35.43619,-83.359952,Monarch Tagged,6
11614,3/30/1998,US,TX,32.50070,-94.740489,Monarch Tagged,1
11618,3/4/1998,US,TX,27.80058,-97.396381,Monarch Tagged,1


In [146]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1998_3_df['decimalLatitude'] = MWatch_Tagged_1998_3_df['decimalLatitude'].astype(float)
MWatch_Tagged_1998_3_df['decimalLongitude'] = MWatch_Tagged_1998_3_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [147]:
geojson = df_to_geojson(MWatch_Tagged_1998_3_df, cols)

In [148]:
file = open("MWatch_Tagged_1998_3.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## April

In [149]:
MWatch_Tagged_1998_month_df =pd.DataFrame(MWatch_Tagged_1998_df)
MWatch_Tagged_1998_4_df = MWatch_Tagged_1998_month_df[MWatch_Tagged_1998_month_df['month']==4]
MWatch_Tagged_1998_4_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11720,4/3/1998,US,TX,33.57927,-95.392175,Monarch Tagged,1,4
11724,4/30/1998,US,TX,28.80527,-97.003598,Monarch Tagged,1,4
11727,4/4/1998,US,TX,33.57927,-95.392175,Monarch Tagged,1,4
11735,4/7/1998,US,FL,28.53834,-81.379237,Monarch Tagged,2,4
11740,4/9/1998,US,TX,32.50070,-94.740489,Monarch Tagged,2,4


In [150]:
MWatch_Tagged_1998_4_df.shape

(19, 8)

In [151]:
MWatch_Tagged_1998_4_df = pd.DataFrame(MWatch_Tagged_1998_4_df)
MWatch_Tagged_1998_4_df.drop(['month'], axis=1, inplace=True)

In [152]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1998_4_df['decimalLatitude'] = MWatch_Tagged_1998_4_df['decimalLatitude'].astype(float)
MWatch_Tagged_1998_4_df['decimalLongitude'] = MWatch_Tagged_1998_4_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [153]:
geojson = df_to_geojson(MWatch_Tagged_1998_4_df, cols)

In [154]:
file = open("MWatch_Tagged_1998_4.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## May

In [155]:
MWatch_Tagged_1998_month_df =pd.DataFrame(MWatch_Tagged_1998_df)
MWatch_Tagged_1998_5_df = MWatch_Tagged_1998_month_df[MWatch_Tagged_1998_month_df['month']==5]
MWatch_Tagged_1998_5_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11830,5/6/1998,US,FL,28.53834,-81.379237,Monarch Tagged,10,5
11831,5/6/1998,US,TX,28.80527,-97.003598,Monarch Tagged,3,5
11832,5/6/1998,US,TX,30.63326,-97.677984,Monarch Tagged,1,5
11839,5/8/1998,US,TX,30.63326,-97.677984,Monarch Tagged,4,5
11843,5/9/1998,US,TX,28.80527,-97.003598,Monarch Tagged,1,5


In [156]:
MWatch_Tagged_1998_5_df.shape

(15, 8)

In [157]:
MWatch_Tagged_1998_5_df =pd.DataFrame(MWatch_Tagged_1998_5_df)
MWatch_Tagged_1998_5_df.drop(['month'], axis=1, inplace=True)

In [158]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1998_5_df['decimalLatitude'] = MWatch_Tagged_1998_5_df['decimalLatitude'].astype(float)
MWatch_Tagged_1998_5_df['decimalLongitude'] = MWatch_Tagged_1998_5_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [159]:
geojson = df_to_geojson(MWatch_Tagged_1998_5_df, cols)

In [160]:
file = open("MWatch_Tagged_1998_5.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## June

In [161]:
MWatch_Tagged_1998_month_df =pd.DataFrame(MWatch_Tagged_1998_df)
MWatch_Tagged_1998_6_df = MWatch_Tagged_1998_month_df[MWatch_Tagged_1998_month_df['month']==6]
MWatch_Tagged_1998_6_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11857,6/13/1998,US,TN,35.72814,-83.813231,Monarch Tagged,1,6
11860,6/14/1998,US,LA,30.45828,-91.140320,Monarch Tagged,3,6
11861,6/14/1998,US,TN,35.72814,-83.813231,Monarch Tagged,1,6
11867,6/16/1998,US,LA,30.45828,-91.140320,Monarch Tagged,1,6


In [162]:
MWatch_Tagged_1998_6_df.shape

(4, 8)

In [163]:
MWatch_Tagged_1998_6_df =pd.DataFrame(MWatch_Tagged_1998_6_df)
MWatch_Tagged_1998_6_df.drop(['month'], axis=1, inplace=True)

In [164]:
MWatch_Tagged_1998_6_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number
11857,6/13/1998,US,TN,35.72814,-83.813231,Monarch Tagged,1
11860,6/14/1998,US,LA,30.45828,-91.140320,Monarch Tagged,3
11861,6/14/1998,US,TN,35.72814,-83.813231,Monarch Tagged,1
11867,6/16/1998,US,LA,30.45828,-91.140320,Monarch Tagged,1


In [165]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1998_6_df['decimalLatitude'] = MWatch_Tagged_1998_6_df['decimalLatitude'].astype(float)
MWatch_Tagged_1998_6_df['decimalLongitude'] = MWatch_Tagged_1998_6_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [166]:
geojson = df_to_geojson(MWatch_Tagged_1998_6_df, cols)

In [167]:
file = open("MWatch_Tagged_1998_6.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## July

In [168]:
MWatch_Tagged_1998_month_df =pd.DataFrame(MWatch_Tagged_1998_df)
MWatch_Tagged_1998_7_df = MWatch_Tagged_1998_month_df[MWatch_Tagged_1998_month_df['month']==7]
MWatch_Tagged_1998_7_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11931,7/29/1998,US,CT,41.27055,-72.946971,Monarch Tagged,4,7
11932,7/29/1998,US,MI,42.49253,-83.697996,Monarch Tagged,1,7
11938,7/30/1998,US,MI,42.49253,-83.697996,Monarch Tagged,1,7
11939,7/30/1998,US,PA,40.48480,-78.010281,Monarch Tagged,24,7
11942,7/31/1998,US,PA,40.48480,-78.010281,Monarch Tagged,13,7


In [169]:
MWatch_Tagged_1998_7_df.shape

(13, 8)

In [170]:
MWatch_Tagged_1998_7_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11908,7/16/1998,US,KS,38.24640,-97.348923,Monarch Tagged,48,7
11912,7/19/1998,US,CT,41.27055,-72.946971,Monarch Tagged,1,7
11914,7/20/1998,US,CT,41.27055,-72.946971,Monarch Tagged,2,7
11916,7/21/1998,US,CT,41.27055,-72.946971,Monarch Tagged,3,7
11917,7/22/1998,US,CT,41.27055,-72.946971,Monarch Tagged,1,7


In [171]:
MWatch_Tagged_1998_7_df =pd.DataFrame(MWatch_Tagged_1998_7_df)
MWatch_Tagged_1998_7_df.drop(['month'], axis=1, inplace=True)

In [172]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1998_7_df['decimalLatitude'] = MWatch_Tagged_1998_7_df['decimalLatitude'].astype(float)
MWatch_Tagged_1998_7_df['decimalLongitude'] = MWatch_Tagged_1998_7_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [173]:
MWatch_Tagged_1998_7_df.head()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number
11908,7/16/1998,US,KS,38.24640,-97.348923,Monarch Tagged,48
11912,7/19/1998,US,CT,41.27055,-72.946971,Monarch Tagged,1
11914,7/20/1998,US,CT,41.27055,-72.946971,Monarch Tagged,2
11916,7/21/1998,US,CT,41.27055,-72.946971,Monarch Tagged,3
11917,7/22/1998,US,CT,41.27055,-72.946971,Monarch Tagged,1


In [174]:
geojson = df_to_geojson(MWatch_Tagged_1998_7_df, cols)

In [175]:
file = open("MWatch_Tagged_1998_7.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## August

In [176]:
MWatch_Tagged_1998_month_df =pd.DataFrame(MWatch_Tagged_1998_df)
MWatch_Tagged_1998_8_df = MWatch_Tagged_1998_month_df[MWatch_Tagged_1998_month_df['month']==8]
MWatch_Tagged_1998_8_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
16130,8/9/1998,US,MI,43.69808,-85.483656,Monarch Tagged,1,8
16131,8/9/1998,US,NY,42.85896,-76.337987,Monarch Tagged,1,8
16132,8/9/1998,US,PA,40.03788,-76.305514,Monarch Tagged,1,8
16133,8/9/1998,US,PA,40.37718,-76.644440,Monarch Tagged,6,8
16134,8/9/1998,US,WI,43.01168,-88.231481,Monarch Tagged,4,8


In [177]:
MWatch_Tagged_1998_8_df.shape

(659, 8)

In [178]:
MWatch_Tagged_1998_8_df = pd.DataFrame(MWatch_Tagged_1998_8_df)
MWatch_Tagged_1998_8_df.drop(['month'], axis=1, inplace=True)

In [179]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1998_8_df['decimalLatitude'] = MWatch_Tagged_1998_8_df['decimalLatitude'].astype(float)
MWatch_Tagged_1998_8_df['decimalLongitude'] = MWatch_Tagged_1998_8_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [180]:
geojson = df_to_geojson(MWatch_Tagged_1998_8_df, cols)

In [181]:
file = open("MWatch_Tagged_1998_8.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## September

In [182]:
MWatch_Tagged_1998_month_df =pd.DataFrame(MWatch_Tagged_1998_df)
MWatch_Tagged_1998_9_df = MWatch_Tagged_1998_month_df[MWatch_Tagged_1998_month_df['month']==9]
MWatch_Tagged_1998_9_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
39697,9/9/1998,US,WI,43.944130,-90.812912,Monarch Tagged,2,9
39698,9/9/1998,US,WI,44.185820,-88.462609,Monarch Tagged,32,9
39699,9/9/1998,US,WI,44.272210,-88.338995,Monarch Tagged,1,9
39700,9/9/1998,US,WI,44.740810,-89.063165,Monarch Tagged,2,9
39701,9/9/1998,US,WV,37.801788,-80.445630,Monarch Tagged,1,9


In [183]:
MWatch_Tagged_1998_9_df.shape

(3922, 8)

In [184]:
MWatch_Tagged_1998_9_df = pd.DataFrame(MWatch_Tagged_1998_9_df)
MWatch_Tagged_1998_9_df.drop(['month'], axis=1, inplace=True)

In [185]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1998_9_df['decimalLatitude'] = MWatch_Tagged_1998_9_df['decimalLatitude'].astype(float)
MWatch_Tagged_1998_9_df['decimalLongitude'] = MWatch_Tagged_1998_9_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [186]:
geojson = df_to_geojson(MWatch_Tagged_1998_9_df, cols)

In [187]:
file = open("MWatch_Tagged_1998_9.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## October

In [188]:
MWatch_Tagged_1998_month_df =pd.DataFrame(MWatch_Tagged_1998_df)
MWatch_Tagged_1998_10_df = MWatch_Tagged_1998_month_df[MWatch_Tagged_1998_month_df['month']==10]
MWatch_Tagged_1998_10_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
10289,10/9/1998,US,TX,36.269990,-100.794160,Monarch Tagged,4,10
10290,10/9/1998,US,VA,37.933180,-75.378809,Monarch Tagged,4,10
10291,10/9/1998,US,WI,44.185820,-88.462609,Monarch Tagged,1,10
10292,10/9/1998,US,WV,37.801788,-80.445630,Monarch Tagged,1,10
10293,10/9/1998,US,WV,38.925940,-79.846735,Monarch Tagged,9,10


In [189]:
MWatch_Tagged_1998_10_df.shape

(1650, 8)

In [190]:
MWatch_Tagged_1998_10_df = pd.DataFrame(MWatch_Tagged_1998_10_df)
MWatch_Tagged_1998_10_df.drop(['month'], axis=1, inplace=True)

In [191]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1998_10_df['decimalLatitude'] = MWatch_Tagged_1998_10_df['decimalLatitude'].astype(float)
MWatch_Tagged_1998_10_df['decimalLongitude'] = MWatch_Tagged_1998_10_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [192]:
geojson = df_to_geojson(MWatch_Tagged_1998_10_df, cols)

In [193]:
file = open("MWatch_Tagged_1998_10.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## November

In [194]:
MWatch_Tagged_1998_month_df =pd.DataFrame(MWatch_Tagged_1998_df)
MWatch_Tagged_1998_11_df = MWatch_Tagged_1998_month_df[MWatch_Tagged_1998_month_df['month']==11]
MWatch_Tagged_1998_11_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11290,11/9/1998,US,FL,28.67000,-81.208120,Monarch Tagged,6,11
11291,11/9/1998,US,NY,42.00509,-74.265980,Monarch Tagged,1,11
11292,11/9/1998,US,TX,27.80058,-97.396381,Monarch Tagged,9,11
11293,11/9/1998,US,TX,28.80527,-97.003598,Monarch Tagged,6,11
11294,11/9/1998,US,TX,29.16941,-95.431885,Monarch Tagged,3,11


In [195]:
MWatch_Tagged_1998_11_df.shape

(157, 8)

In [196]:
MWatch_Tagged_1998_11_df = pd.DataFrame(MWatch_Tagged_1998_11_df)
MWatch_Tagged_1998_11_df.drop(['month'], axis=1, inplace=True)

In [197]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1998_11_df['decimalLatitude'] = MWatch_Tagged_1998_11_df['decimalLatitude'].astype(float)
MWatch_Tagged_1998_11_df['decimalLongitude'] = MWatch_Tagged_1998_11_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [198]:
geojson = df_to_geojson(MWatch_Tagged_1998_11_df, cols)

In [199]:
file = open("MWatch_Tagged_1998_11.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()

## December

In [200]:
MWatch_Tagged_1998_month_df =pd.DataFrame(MWatch_Tagged_1998_df)
MWatch_Tagged_1998_12_df = MWatch_Tagged_1998_month_df[MWatch_Tagged_1998_month_df['month']==12]
MWatch_Tagged_1998_12_df.tail()

,eventDate,country,stateProvince,decimalLatitude,decimalLongitude,species,number,month
11502,12/7/1998,US,TX,29.50745,-95.094930,Monarch Tagged,3,12
11509,12/8/1998,US,LA,29.95107,-90.071532,Monarch Tagged,14,12
11510,12/8/1998,US,TX,27.80058,-97.396381,Monarch Tagged,2,12
11511,12/8/1998,US,TX,29.50745,-95.094930,Monarch Tagged,1,12
11526,12/9/1998,US,LA,29.95107,-90.071532,Monarch Tagged,2,12


In [201]:
MWatch_Tagged_1998_12_df.shape

(44, 8)

In [202]:
MWatch_Tagged_1998_12_df = pd.DataFrame(MWatch_Tagged_1998_12_df)
MWatch_Tagged_1998_12_df.drop(['month'], axis=1, inplace=True)

In [203]:
# convert lat-long to floats and change address from ALL CAPS to regular capitalization
MWatch_Tagged_1998_12_df['decimalLatitude'] = MWatch_Tagged_1998_12_df['decimalLatitude'].astype(float)
MWatch_Tagged_1998_12_df['decimalLongitude'] = MWatch_Tagged_1998_12_df['decimalLongitude'].astype(float)
# JNorth_Milkweed_Site_df['street_address'] = JNorth_Milkweed_Site_df['street_address'].str.title()

In [204]:
geojson = df_to_geojson(MWatch_Tagged_1998_12_df, cols)

In [205]:
file = open("MWatch_Tagged_1998_12.geojson", "w")
json.dump(geojson, file, indent =1)
file.close()